In [ ]:
# ============================================================
# Imports
# ============================================================
import os
import glob
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, RidgeCV
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.interpolate import interp1d, PchipInterpolator
import matplotlib.pyplot as plt
from statsmodels.robust.scale import mad


In [ ]:
# ============================================================
# Load input data (replace file paths with your dataset)
# ============================================================
# NACC FreeSurfer combined dataset
nacc_tbl = pd.read_csv("NACC_FS_combinedVolumeThickness.csv")

# NACC outlier flags
nacc_nl_ols = pd.read_csv("NACC_outliers.csv")

# Load ADNI dataset (replace file paths with your dataset)
adni_tbl = pd.read_csv("ADNI_FS_combinedVolumeThickness.csv")

# Create flags for baseline visits and non-outliers
flag_inc = (nacc_nl_ols['ol_count'] == 0) & (nacc_nl_ols['V1'] == 1)


# Load additional ADNI Excel sheets (replace file paths)
adni1 = pd.read_excel("ADNI_All.xlsx", sheet_name="Sheet1")
adni2 = pd.read_excel("ADNI_All.xlsx", sheet_name="Subcortical Volume")
adni3 = pd.read_excel("ADNI_All.xlsx", sheet_name="Cortical Area lh")
adni4 = pd.read_excel("ADNI_All.xlsx", sheet_name="Cortical Area rh")
adni5 = pd.read_excel("ADNI_All.xlsx", sheet_name="Cortical Thickness lh")
adni6 = pd.read_excel("ADNI_All.xlsx", sheet_name="Cortical Thickness rh")
adni7 = pd.read_excel("ADNI_All.xlsx", sheet_name="Cortical Vol lh")
adni8 = pd.read_excel("ADNI_All.xlsx", sheet_name="Cortical Vol rh")



In [ ]:
# ============================================================
#Outlier Detection
# ============================================================

# Compute mean and standard deviation vectors for features
meanvector = np.zeros(nacc_tbl.shape[1])
meanvector[4:] = nacc_tbl.iloc[:, 4:].mean().values

sdvector = np.zeros(nacc_tbl.shape[1])
sdvector[4:] = nacc_tbl.iloc[:, 4:].std().values

# Compute z-scores for feature columns (5:189)
nacc_tblz = nacc_tbl.copy()
zscore_block = (nacc_tbl.iloc[:, 4:] - meanvector[4:]) / sdvector[4:]
for i, col in enumerate(nacc_tbl.columns[4:]):
    nacc_tblz[col] = zscore_block.iloc[:, i].astype(float)

# Mark outliers using 4 SD threshold
outlierThreshold = 4
outliers = np.zeros_like(nacc_tblz, dtype=bool)
outliers[:, 4:] = np.abs(nacc_tblz.iloc[:, 4:]) > outlierThreshold

# LOESS smoothing for each feature vs age (NACC dataset)
Vage = np.arange(18, 100.1, 0.1)
traj_tbl = pd.DataFrame(np.zeros((len(Vage), 185)))
sd_tbl = np.zeros(185)
RsquareAge_tbl = np.zeros(185)
nacc_tblz_adj = nacc_tblz.copy()

for i in range(185):
    mask = (outliers[:, i + 4] == False) & flag_inc.values
    x = nacc_tbl.iloc[:, 3][mask]  # Age
    y = nacc_tbl.iloc[:, i + 4][mask]  # Measure
    
    sorted_idx = np.argsort(x)
    sortx, sorty = x.iloc[sorted_idx], y.iloc[sorted_idx]
    
    loess_result = lowess(sorty, sortx, frac=0.75, return_sorted=True)
    sortx_unique, idx_unique = np.unique(loess_result[:, 0], return_index=True)
    ysmooth_unique = loess_result[idx_unique, 1]
    
    traj_tbl.iloc[:, i] = PchipInterpolator(sortx_unique, ysmooth_unique)(Vage)
    sd_tbl[i] = np.std(sorty - PchipInterpolator(sortx_unique, ysmooth_unique)(sortx))
    
    xfull = nacc_tbl["Age"]
    yfull = nacc_tbl.iloc[:, i + 4]
    yhat = PchipInterpolator(sortx_unique, ysmooth_unique)(xfull)
    RsquareAge_tbl[i] = 1 - np.var(yfull - yhat) / np.var(yfull)
    
    Zadj = (yfull - yhat) / sd_tbl[i]
    nacc_tblz_adj.iloc[:, i + 4] = Zadj

# Update outliers based on adjusted z-scores
outliersraw = outliers.copy()
outliers[:, 4:] = np.abs(nacc_tblz_adj.iloc[:, 4:]) > outlierThreshold

# Validity mask
valid = np.sum(outliers, axis=1) == 0



In [ ]:
# ============================================================
# Per-feature Prediction 
# ============================================================

# LOESS smoothing of age vs measure (cross-sectional curves)
zscorevector = np.arange(-4, 4.01, 0.01)
loess_tbl_measure = pd.DataFrame(np.zeros((len(zscorevector), 185)))
loess_tbl_age = pd.DataFrame(np.zeros((len(zscorevector), 185)))

for i in range(185):
    mask = flag_inc.values & (~outliers[:, i + 4])
    x = nacc_tblz.iloc[:, i + 4][mask]
    y = nacc_tblz.iloc[:, 3][mask]
    
    sorted_idx = np.argsort(x)
    sortx, sorty = x.iloc[sorted_idx], y.iloc[sorted_idx]
    
    loess_result = lowess(sorty, sortx, frac=0.6, return_sorted=True)
    loess_tbl_measure.iloc[:, i] = zscorevector * sdvector[i + 4] + meanvector[i + 4]
    
    sortx_unique, idx_unique = np.unique(loess_result[:, 0], return_index=True)
    ysmooth_unique = loess_result[idx_unique, 1]
    loess_tbl_age.iloc[:, i] = PchipInterpolator(sortx_unique, ysmooth_unique)(zscorevector)


# Predict age from LOESS curves (NACC dataset)
age_pred_tbl = pd.DataFrame(np.zeros((nacc_tbl.shape[0], 185)))
for i in range(185):
    interp_func = interp1d(
        loess_tbl_measure.iloc[:, i],
        loess_tbl_age.iloc[:, i],
        kind='linear',
        fill_value='extrapolate'
    )
    pred_age = interp_func(nacc_tbl.iloc[:, i + 4])
    pred_age = np.clip(pred_age, 0, 110)
    age_pred_tbl.iloc[:, i] = pred_age



In [ ]:
# ============================================================
# Predict local age and compute adjusted z-scores (ADNI dataset)
# ============================================================
age_local_adni = adni_tbl.iloc[:, 4:189].copy()
adni_tblz_adj = adni_tbl.copy()

for i in range(185):
    interp_func = interp1d(
        loess_tbl_measure.iloc[:, i],
        loess_tbl_age.iloc[:, i],
        kind='linear',
        fill_value='extrapolate'
    )
    pred_age_local = interp_func(adni_tbl.iloc[:, i + 4])
    pred_age_local = np.clip(pred_age_local, 0, 110)
    age_local_adni.iloc[:, i] = pred_age_local
    
    xfull = adni_tbl["Age"]
    yfull = adni_tbl.iloc[:, i + 4]
    yhat = PchipInterpolator(Vage, traj_tbl.iloc[:, i])(xfull)
    Zadj = (yfull - yhat) / sd_tbl[i]
    adni_tblz_adj.iloc[:, i + 4] = Zadj

outliers_adni = np.zeros_like(adni_tblz_adj, dtype=bool)
outliers_adni[:, 4:] = np.abs(adni_tblz_adj.iloc[:, 4:]) > outlierThreshold
valid_adni = np.sum(outliers_adni, axis=1) == 0


In [ ]:
# ============================================================
# Train Ridge regression model (cross-validated) on NACC
# ============================================================
X_nacc = age_pred_tbl[valid].values
y_nacc = nacc_tbl[valid]['Age'].values

ridge = RidgeCV(cv=10)
ridge.fit(X_nacc, y_nacc)


In [ ]:
# ============================================================
# Evaluate model on ADNI dataset
# ============================================================
y_adni_pred = ridge.predict(age_local_adni.values)
y_adni_true = adni1["Age"].values


In [ ]:
# ============================================================
# Analyze top contributing features (Ridge coefficients)
# ============================================================
x = 20
coef_abs = np.abs(ridge.coef_)
top_idx = np.argsort(coef_abs)[-x:][::-1]

feature_names = nacc_tbl.columns[4:189]
top_feature_names = feature_names[top_idx]

plt.figure(figsize=(10, 5))
plt.bar(range(x), coef_abs[top_idx])
plt.xticks(range(x), top_feature_names, rotation=45, ha='right')
plt.xlabel('Feature Index')
plt.ylabel('Absolute Coefficient Value')
plt.title(f'Top {x} Absolute Ridge Coefficients')
plt.show()

top_coef_table = pd.DataFrame({
    'Feature': top_feature_names,
    'Coefficient': ridge.coef_[top_idx],
    'Absolute Coefficient': coef_abs[top_idx]
})
print(top_coef_table)


In [ ]:
# ============================================================
# Compute MAE per age bucket (NACC and ADNI)
# ============================================================
y_nacc_pred = ridge.predict(X_nacc)
y_adni_pred = ridge.predict(age_local_adni.values)
y_adni_true = adni_tbl["Age"].values

bins = [18, 30, 40, 50, 60, 70, 80, 90, 100, 110]
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]

nacc_bucket = pd.cut(y_nacc, bins=bins, labels=labels, right=False)
adni_bucket = pd.cut(y_adni_true[valid_adni], bins=bins, labels=labels, right=False)

def mae_per_bucket(y_true, y_pred, bucket):
    mae_list = []
    for label in labels:
        mask = bucket == label
        mae = np.mean(np.abs(y_true[mask] - y_pred[mask])) if np.any(mask) else np.nan
        mae_list.append(mae)
    return mae_list

mae_nacc = mae_per_bucket(y_nacc, y_nacc_pred, nacc_bucket)
mae_adni = mae_per_bucket(y_adni_true[valid_adni], y_adni_pred[valid_adni], adni_bucket)
overall_mae_nacc = np.mean(np.abs(y_nacc - y_nacc_pred))
overall_mae_adni = np.mean(np.abs(y_adni_true[valid_adni] - y_adni_pred[valid_adni]))

results = pd.DataFrame({
    'Age Bucket': labels,
    'NACC MAE': mae_nacc,
    'ADNI MAE': mae_adni
})
results.loc[len(results)] = ['Overall', overall_mae_nacc, overall_mae_adni]
print(results)


In [ ]:
# ============================================================
# Estimate within-group variance from repeated measures (NACC)
# ============================================================
valid_nacc_tbl = nacc_tbl[valid].reset_index(drop=True)
y_valid_pred = y_nacc_pred

grouped = valid_nacc_tbl.groupby(['NACCID', 'Age'])
num_groups_gt2 = sum(len(idx) >= 2 for idx in grouped.groups.values())

AgeDiff_list = []
for _, idx in grouped.groups.items():
    if len(idx) > 1:
        if len(idx) > 2:
            idx1, idx2 = np.random.choice(idx, 2, replace=False)
        else:
            idx1, idx2 = idx[0], idx[1]
        age_diff = np.abs(y_valid_pred[idx1] - y_valid_pred[idx2])
        AgeDiff_list.append(age_diff)

age_diff_variance = np.var(AgeDiff_list) / 2
age_diff_std = np.sqrt(age_diff_variance)

print("Standard Deviation of Age Differences:", age_diff_std)
